In [1]:
# hyperparameter tuning

# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os
import dagshub


In [2]:
mlflow.set_tracking_uri('https://dagshub.com/arshadjamal6002/mlops-emotion-detection.mlflow')
dagshub.init(repo_owner='arshadjamal6002', repo_name='mlops-emotion-detection', mlflow=True)

# Load the data
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])


Accessing as arshadjamal6002

Initialized MLflow to track repo "arshadjamal6002/mlops-emotion-detection"

Repository arshadjamal6002/mlops-emotion-detection initialized!

In [3]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise


In [4]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]
df.head()

,sentiment,content
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
6,sadness,"I should be sleep, but im not! thinking about ..."
8,sadness,@charviray Charlene my love. I miss you
9,sadness,@kelcouch I'm sorry at least it's Friday?


In [5]:
# Normalize the text data
df = normalize_text(df)

df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

C:\Users\arsha\AppData\Local\Temp\ipykernel_11820\4210321288.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


In [6]:
# Set the experiment name
mlflow.set_experiment("LoR Hyperparameter Tuning")


<Experiment: artifact_location='mlflow-artifacts:/8d6acf7c2fbb4248b70d4d3e1bd61f5e', creation_time=1731400041084, experiment_id='2', last_update_time=1731400041084, lifecycle_stage='active', name='LoR Hyperparameter Tuning', tags={}>

In [ ]:
# Define hyperparameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10],                # Regularization strength
    'penalty': ['l1', 'l2'],          # Regularization type (L1 or L2)
    'solver': ['liblinear', 'saga'],   # Solver algorithms
    'max_iter' : [1000]
}


In [10]:
# Start the parent run for hyperparameter tuning
with mlflow.start_run():

    # Perform grid search
    grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Log each parameter combination as a child run
    for params, mean_score, std_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['std_test_score']):
        with mlflow.start_run(run_name=f"LR with params: {params}", nested=True):
            model = LogisticRegression(**params)
            model.fit(X_train, y_train)
            
            # Model evaluation
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            
            # Log parameters and metrics
            mlflow.log_params(params)
            mlflow.log_metric("mean_cv_score", mean_score)
            mlflow.log_metric("std_cv_score", std_score)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            
            
            # Print the results for verification
            print(f"Mean CV Score: {mean_score}, Std CV Score: {std_score}")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1}")

    # Log the best run details in the parent run
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    mlflow.log_params(best_params)
    mlflow.log_metric("best_f1_score", best_score)
    
    print(f"Best Params: {best_params}")
    print(f"Best F1 Score: {best_score}")

    # Save and log the notebook
    mlflow.log_artifact(__file__)

    # Log model
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model")

    # Save DataFrame as CSV
    df.to_csv("logged_data.csv", index=False)
    # Log the CSV as an artifact
    mlflow.log_artifact("logged_data.csv")

c:\Users\arsha\Downloads\sentiment-detection-new\mlops-emotion-detection\myenv\lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Mean CV Score: 0.7050121790562888, Std CV Score: 0.014034901421846454
Accuracy: 0.739277108433735
Precision: 0.7749419953596288
Recall: 0.6581280788177339
F1 Score: 0.7117741076185402


2024/11/12 11:55:14 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'} at: https://dagshub.com/arshadjamal6002/mlops-emotion-detection.mlflow/#/experiments/2/runs/89a7f5e81fe64865b34be581680f451c.
2024/11/12 11:55:14 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/arshadjamal6002/mlops-emotion-detection.mlflow/#/experiments/2.


Mean CV Score: 0.7041946418702394, Std CV Score: 0.013441626884284798
Accuracy: 0.740722891566265
Precision: 0.7809187279151943
Recall: 0.6532019704433497
F1 Score: 0.7113733905579399


2024/11/12 11:55:19 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'} at: https://dagshub.com/arshadjamal6002/mlops-emotion-detection.mlflow/#/experiments/2/runs/7c31f38a15cf421ba8e738faace513aa.
2024/11/12 11:55:19 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/arshadjamal6002/mlops-emotion-detection.mlflow/#/experiments/2.


...

Mean CV Score: 0.7863215938536148, Std CV Score: 0.007852345486681426
Accuracy: 0.7889156626506024
Precision: 0.778743961352657
Recall: 0.7940886699507389
F1 Score: 0.7863414634146342


2024/11/12 11:57:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 10, 'penalty': 'l2', 'solver': 'saga'} at: https://dagshub.com/arshadjamal6002/mlops-emotion-detection.mlflow/#/experiments/2/runs/bc5b744fa09347f4b6f2850d42f4efc8.
2024/11/12 11:57:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/arshadjamal6002/mlops-emotion-detection.mlflow/#/experiments/2.


Best Params: {'C': 1, 'penalty': 'l2', 'solver': 'saga'}
Best F1 Score: 0.7897162715579451


2024/11/12 11:57:19 INFO mlflow.tracking._tracking_service.client: 🏃 View run agreeable-wasp-515 at: https://dagshub.com/arshadjamal6002/mlops-emotion-detection.mlflow/#/experiments/2/runs/e2688f7950594f5c8b06dc64c79c81b9.
2024/11/12 11:57:19 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/arshadjamal6002/mlops-emotion-detection.mlflow/#/experiments/2.


NameError: name '__file__' is not defined